In [75]:
from datetime import datetime
import sys
from SPARQLWrapper import SPARQLWrapper, JSON
from PIL import Image
from markdownify import markdownify 
import folium
import io




# endpoint_url = "https://computational-publishing-service.wikibase.cloud/query/sparql"

endpoint_url = "https://query.wikidata.org/sparql"

query = """#defaultView:Map

SELECT DISTINCT ?item ?itemLabel ?latitude ?longitude ?geo ?locationLabel WHERE {
 VALUES ?type { wd:Q1007870 wd:Q33506 wd:Q2668072 wd:Q7075 wd:Q166118 wd:Q56190453 }
  #art gallery, museum, collection, library, archive, memorial
  
  ?item wdt:P31* ?type; # Must be an instance of a relevant cultural type
        (wdt:P276|wdt:P131)* wd:Q1715.  # Located in Hannover or a part of Hannover (or a part of a part of Hannover...) 
  
  OPTIONAL { ?item wdt:P625 ?geo }
  OPTIONAL { ?item wdt:P131 ?location } 
  # Get coordinates & location if they exist

  SERVICE wikibase:label { bd:serviceParam wikibase:language "de,[AUTO_LANGUAGE]". }

  BIND(geof:latitude(?geo) AS ?latitude).
  BIND(geof:longitude(?geo) AS ?longitude)
}

ORDER BY DESC(?geo)
"""



def run_query(endpoint_url, query):
    user_agent = "WDQS-example Python/%s.%s" % (sys.version_info[0], sys.version_info[1])
    sparql = SPARQLWrapper(endpoint_url, agent=user_agent)
    sparql.setQuery(query)
    sparql.setMethod("POST") #this NEEDS to be added to get results (not included in the wikibase python example code)
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()

locations = []

def get_items():

    q = query

    results = run_query(endpoint_url, q)

    for item in results["results"]["bindings"]:    

      alt='image of a ' + item['itemDescription']['value'] if 'itemDescription' in item else ''
      title=item['itemLabel']['value'] if 'itemLabel' in item else ''
      year=item['year']['value'] if 'year' in item else ''
      latitude=item['latitude']['value'] if 'latitude' in item else ''
      longitude=item['longitude']['value'] if 'longitude' in item else ''
      place=item['locationLabel']['value'] if 'locationLabel' in item else ''
      item_url=item['item']['value'] if 'item' in item else ''
      image_url=item['image']['value'] if 'image' in item else ''

      locations.append((title, latitude, longitude, image_url, item_url, place, alt))






In [76]:
get_items()

m_hannover = folium.Map(location=[52.38, 9.74], zoom_start=12)

for name, lat, lon, url, item, place, alt in locations:
    if not lat or not lon:  # Überspringen, wenn lat oder lon leer sind
        continue
    
    folium.Marker(
        location=[lat, lon],
        popup=f"""<h3><a href='{item}' target='_blank'>{name}</a></h3>
        {place}""",
        tooltip=name  # Hover label
    ).add_to(m_hannover)

print("""
[Wikidata Link](https://query.wikidata.org/embed.html#%23defaultView%3AMap%0A%0ASELECT%20DISTINCT%20%3Fitem%20%3FitemLabel%20%3Flatitude%20%3Flongitude%20%3Fgeo%20%3FlocationLabel%20WHERE%20%7B%0A%20VALUES%20%3Ftype%20%7B%20wd%3AQ1007870%20wd%3AQ33506%20wd%3AQ2668072%20wd%3AQ7075%20wd%3AQ166118%20wd%3AQ56190453%20%7D%0A%20%20%23art%20gallery%2C%20museum%2C%20collection%2C%20library%2C%20archive%2C%20memorial%0A%20%20%0A%20%20%3Fitem%20wdt%3AP31%2a%20%3Ftype%3B%20%23%20Must%20be%20an%20instance%20of%20a%20relevant%20cultural%20type%0A%20%20%20%20%20%20%20%20%28wdt%3AP276%7Cwdt%3AP131%29%2a%20wd%3AQ1715.%20%20%23%20Located%20in%20Hannover%20or%20a%20part%20of%20Hannover%20%28or%20a%20part%20of%20a%20part%20of%20Hannover...%29%20%0A%20%20%0A%20%20OPTIONAL%20%7B%20%3Fitem%20wdt%3AP625%20%3Fgeo%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fitem%20wdt%3AP131%20%3Flocation%20%7D%20%0A%20%20%23%20Get%20coordinates%20%26%20location%20if%20they%20exist%0A%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3AserviceParam%20wikibase%3Alanguage%20%22de%2C%5BAUTO_LANGUAGE%5D%22.%20%7D%0A%0A%20%20BIND%28geof%3Alatitude%28%3Fgeo%29%20AS%20%3Flatitude%29.%0A%20%20BIND%28geof%3Alongitude%28%3Fgeo%29%20AS%20%3Flongitude%29%0A%7D%0A%0AORDER%20BY%20DESC%28%3Fgeo%29%0A%0A)
""")


[Wikidata Link](https://query.wikidata.org/embed.html#%23defaultView%3AMap%0A%0ASELECT%20DISTINCT%20%3Fitem%20%3FitemLabel%20%3Flatitude%20%3Flongitude%20%3Fgeo%20%3FlocationLabel%20WHERE%20%7B%0A%20VALUES%20%3Ftype%20%7B%20wd%3AQ1007870%20wd%3AQ33506%20wd%3AQ2668072%20wd%3AQ7075%20wd%3AQ166118%20wd%3AQ56190453%20%7D%0A%20%20%23art%20gallery%2C%20museum%2C%20collection%2C%20library%2C%20archive%2C%20memorial%0A%20%20%0A%20%20%3Fitem%20wdt%3AP31%2a%20%3Ftype%3B%20%23%20Must%20be%20an%20instance%20of%20a%20relevant%20cultural%20type%0A%20%20%20%20%20%20%20%20%28wdt%3AP276%7Cwdt%3AP131%29%2a%20wd%3AQ1715.%20%20%23%20Located%20in%20Hannover%20or%20a%20part%20of%20Hannover%20%28or%20a%20part%20of%20a%20part%20of%20Hannover...%29%20%0A%20%20%0A%20%20OPTIONAL%20%7B%20%3Fitem%20wdt%3AP625%20%3Fgeo%20%7D%0A%20%20OPTIONAL%20%7B%20%3Fitem%20wdt%3AP131%20%3Flocation%20%7D%20%0A%20%20%23%20Get%20coordinates%20%26%20location%20if%20they%20exist%0A%0A%20%20SERVICE%20wikibase%3Alabel%20%7B%20bd%3Aser

In [77]:

m_hannover


In [78]:
print("""<table class='table'>
      <thead>
      <tr>
      <th>Name</th>
      <th>Koordinaten</th>
      <th>Ort</th>
      </tr>
      <tbody>
      """)
for name, lat, lon, url, item, place, alt in locations:
  print(f"""
  <tr>
  <td><a href='{item}' style='width:60%'>{name}</a></td>
  """)
  if lat == '' and lon == '':
    print("<td></td>")
  else:
    print(f"<td>{lat},<br> {lon}</td>")
  print(f"""
  <td>{place}</td>
  </tr>
  """)
print("</tbody></table>") 

<table class='table'>
      <thead>
      <tr>
      <th>Name</th>
      <th>Koordinaten</th>
      <th>Ort</th>
      </tr>
      <tbody>
      

  <tr>
  <td><a href='http://www.wikidata.org/entity/Q28738627' style='width:60%'>Bibliothek der Bundesanstalt für Geowissenschaften und Rohstoffe und des Landesamts für Bergbau, Energie und Geologie im Geozentrum Hannover</a></td>
  
<td>52.40406,<br> 9.82038</td>

  <td>Hannover</td>
  </tr>
  

  <tr>
  <td><a href='http://www.wikidata.org/entity/Q76636159' style='width:60%'>Münzkabinett der TUI AG</a></td>
  
<td>52.384001,<br> 9.811558</td>

  <td>Hannover</td>
  </tr>
  

  <tr>
  <td><a href='http://www.wikidata.org/entity/Q126991882' style='width:60%'>Galerie am Tiergarten</a></td>
  
<td>52.361007196,<br> 9.845988302</td>

  <td>Anderten</td>
  </tr>
  

  <tr>
  <td><a href='http://www.wikidata.org/entity/Q64737923' style='width:60%'>Galerie Christoph Kühl</a></td>
  
<td>52.362222222,<br> 9.821944444</td>

  <td>Hannover</td>
  </